# Assignment 3<br>
## 2. Image Classification with Convolution Neural Network (CNN)


### Antoine Wang 260766084

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

#### 1. Use Pytorch class torchvision.datasets.MNIST to (down)load the dataset.Use batch size of 32.

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

trainset = torchvision.datasets.MNIST(root='', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers= 32)

testset = torchvision.datasets.MNIST(root='', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers= 32)

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


#### 2. Implement a CNN with the layers mentioned below. 
- A convolution layer with 32 kernels of size 33.
- A ReLU activation.
- A convolution layer with 64 kernels of size 33.
- A ReLU activation.
- A maxpool layer with kernels of size 22.
- A convolution layer with 64 kernels of size 33.
- A ReLU activation.
- A convolution layer with 64 kernels of size 33.
- A ReLU activation.
- A flattening layer. (This layer resizes 2D feature map to a featurevector. The length of this feature vector should be 4096.)
- A Linear layer with output size of 10.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64 * 8 * 8, 10)


    def forward(self, x):
        # 28*28 per img for MNIST initially
        
        # 1. 32 channel, 26*26 per img
        # 2. RELU
        # 3. 64 channel, 24*24
        # 4. RELU
        x = F.relu(self.conv2(F.relu(self.conv1(x))))
        
        # 5. max pool, 64 channel,12*12
        x = self.pool(x)
        
        # 6. 64c, 10*10
        # 7. relu
        # 8. 64c, 8*8
        # 9. relu
        x = F.relu(self.conv3(F.relu(self.conv3(x))))
        
        # 10. flattening into 64*8*8 = 4096
        x = x.view(-1, 64 * 8 * 8)
       
        # linear layer of 10
        x = self.fc1(x)
        return x


net = Net()

#### 3. Create an instance of SGD optimizer with learning rate of 0.001. Usethe default setting for rest of the hyperparameters. Create an instance of categorical cross entropy criterion. 

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#### 4. Train the CNN for 10 epochs.

There are 60000 images, with 32 image per batch. Thus there will roughly be 1800
batches. Thus in the training process below I choose to print the loss every 200 batches.

In [ ]:
# code credit: ecse 415 tutorial 6
# Modified by AntoineW

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.022
[1,   400] loss: 0.520
[1,   600] loss: 0.378
[1,   800] loss: 0.293
[1,  1000] loss: 0.224
[1,  1200] loss: 0.207
[1,  1400] loss: 0.160
[1,  1600] loss: 0.164
[1,  1800] loss: 0.155
[2,   200] loss: 0.119
[2,   400] loss: 0.123
[2,   600] loss: 0.103
[2,   800] loss: 0.097
[2,  1000] loss: 0.095
[2,  1200] loss: 0.104
[2,  1400] loss: 0.091
[2,  1600] loss: 0.090
[2,  1800] loss: 0.078
[3,   200] loss: 0.078
[3,   400] loss: 0.067
[3,   600] loss: 0.083
[3,   800] loss: 0.077
[3,  1000] loss: 0.071
[3,  1200] loss: 0.066
[3,  1400] loss: 0.060
[3,  1600] loss: 0.064
[3,  1800] loss: 0.064
[4,   200] loss: 0.056
[4,   400] loss: 0.060
[4,   600] loss: 0.062
[4,   800] loss: 0.058
[4,  1000] loss: 0.059
[4,  1200] loss: 0.059
[4,  1400] loss: 0.054
[4,  1600] loss: 0.050
[4,  1800] loss: 0.055
[5,   200] loss: 0.051
[5,   400] loss: 0.044
[5,   600] loss: 0.044
[5,   800] loss: 0.054
[5,  1000] loss: 0.040
[5,  1200] loss: 0.053
[5,  1400] loss: 0.048
[5,  1600] 

In [ ]:
# I did not use GPU thus the speed for traing the network is a bit slow. Thus 
# save the network for later testing
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

#### 5. Predicts labels of the test images using the above trained CNN. Measure and display classification accuracy. 

In [10]:
# loading the training network
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
      # get test img and label
        images, labels = data
        # put the test img into network and get predicted label
        outputs = net(images)
        # choose the mostly likely class as predicted label
        _, predicted = torch.max(outputs.data, 1)

        
        total += labels.size(0)
        # compare predict and label, record correct label
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 98 %
